In [1]:
import torch
import os
import cv2
import numpy as np

import torchvision.transforms as transforms
import sys
sys.path.insert(0, "main")
sys.path.insert(0, "./")

from common.utils.preprocessing import load_img, process_bbox, generate_patch_image
from config import cfg



img_path = 'demo/input.png'
original_img = load_img(img_path)
original_img_height, original_img_width = original_img.shape[:2]

detector = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
with torch.no_grad():
    results = detector(original_img)
    
person_results = results.xyxy[0][results.xyxy[0][:, 5] == 0]
class_ids, confidences, boxes = [], [], []
for detection in person_results:
    x1, y1, x2, y2, confidence, class_id = detection.tolist()
    class_ids.append(class_id)
    confidences.append(confidence)
    boxes.append([x1, y1, x2 - x1, y2 - y1])
indices = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)

persons_deteted = []
for num, indice in enumerate(indices):
    bbox = boxes[indice]  # x,y,h,w
    bbox = process_bbox(bbox, original_img_width, original_img_height)
    img, img2bb_trans, bb2img_trans = generate_patch_image(original_img, bbox, 1.0, 0.0, False, cfg.input_img_shape)
    persons_deteted.append(img)







Using cache found in /home/x_hensh/.cache/torch/hub/ultralytics_yolov5_master


requirements: Ultralytics requirements ['requests>=2.32.2', 'tqdm>=4.66.3', 'setuptools>=70.0.0'] not found, attempting AutoUpdate...


ERROR: Could not install packages due to an OSError: [Errno 30] Read-only file system: '/software/sse/manual/PyTorch/2.3.0/python-3.10/envs/pytorch_2.3.0/lib/python3.10/site-packages/tqdm'



Retry 1/2 failed: Command 'pip install --no-cache-dir "requests>=2.32.2" "tqdm>=4.66.3" "setuptools>=70.0.0" ' returned non-zero exit status 1.


ERROR: Could not install packages due to an OSError: [Errno 30] Read-only file system: '/software/sse/manual/PyTorch/2.3.0/python-3.10/envs/pytorch_2.3.0/lib/python3.10/site-packages/tqdm'



Retry 2/2 failed: Command 'pip install --no-cache-dir "requests>=2.32.2" "tqdm>=4.66.3" "setuptools>=70.0.0" ' returned non-zero exit status 1.
requirements: ❌ Command 'pip install --no-cache-dir "requests>=2.32.2" "tqdm>=4.66.3" "setuptools>=70.0.0" ' returned non-zero exit status 1.


YOLOv5 🚀 2024-12-10 Python-3.10.0 torch-1.13.1+cu117 CUDA:0 (NVIDIA A100-SXM4-80GB, 81051MiB)

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [2]:

sys.path.insert(0, "main/transformer_utils")

from common.base import Demoer
from common.utils.human_models import smpl_x
from common.utils.human_models import smpl_h
from common.utils.human_models import smil_h

transform = transforms.ToTensor()
cfg.set_additional_args(
    encoder_setting='osx_l',
    decoder_setting='wo_face_decoder', 
    pretrained_model_path='pretrained_models/osx_l.pth.tar')

demoerx = Demoer()
demoerx._make_model(smpl_x)
demoerx.model.eval()

demoerh = Demoer()
demoerh._make_model(smpl_h)
demoerh.model.eval()

demoeri = Demoer()
demoeri._make_model(smil_h)
demoeri.model.eval()

outx = []
outh = []
outi = []

for person_detected in persons_deteted:
    img = transform(person_detected.astype(np.float32))/255
    img = img.cuda()[None,:,:,:]
    inputs = {'img': img}

    with torch.no_grad():
        outx.append(demoerx.model(inputs, {}, {}, 'test'))
        outh.append(demoerh.model(inputs, {}, {}, 'test'))
        outi.append(demoeri.model(inputs, {}, {}, 'test'))




        


/home/x_hensh/.local/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)
12-15 23:24:17 Load checkpoint from pretrained_models/osx_l.pth.tar
12-15 23:24:17 Creating graph...
12-15 23:24:28 Load checkpoint from pretrained_models/osx_l.pth.tar
12-15 23:24:28 Load checkpoint from pretrained_models/osx_l.pth.tar
12-15 23:24:28 Creating graph...
12-15 23:24:28 Creating graph...
12-15 23:24:35 Load checkpoint from pretrained_models/osx_l.pth.tar
12-15 23:24:35 Load checkpoint from pretrained_models/osx_l.pth.tar
12-15 23:24:35 Load checkpoint from pretrained_models/osx_l.pth.tar
12-15 23:24:35 Creating graph...
12-15 23:24:35 Creating graph...
12-15 23:24:35 Creating graph...


In [3]:
print (outx[0].keys())
print (outx[0]['outpu'].__dict__.keys())

dict_keys(['img', 'joint_img', 'smplx_joint_proj', 'smplx_mesh_cam', 'smplx_root_pose', 'smplx_body_pose', 'smplx_lhand_pose', 'smplx_rhand_pose', 'smplx_jaw_pose', 'smplx_shape', 'smplx_expr', 'cam_trans', 'lhand_bbox', 'rhand_bbox', 'face_bbox', 'outpu'])
dict_keys(['vertices', 'joints', 'full_pose', 'global_orient', 'transl', 'betas', 'body_pose', 'left_hand_pose', 'right_hand_pose', 'expression', 'jaw_pose'])


In [13]:
for item in outx:
    print (item['outpu'].vertices[0,:,0].cpu().numpy().min(), item['outpu'].vertices[0,:,0].cpu().numpy().max(), \
    item['outpu'].vertices[0,:,1].cpu().numpy().min(), item['outpu'].vertices[0,:,1].cpu().numpy().max(), \
    item['outpu'].vertices[0,:,2].cpu().numpy().min(), item['outpu'].vertices[0,:,2].cpu().numpy().max())

-0.44423175 0.33620498 -0.9853153 0.60636604 -0.5034078 0.34681487
-0.25971022 0.23678397 -1.102833 0.5907675 -0.25650448 0.18536916
-0.44815582 0.32510862 -1.0463173 0.53509444 -0.47288802 0.19826841
-0.27214545 0.2810422 -1.1034721 0.58160144 -0.26713383 0.18028995
-0.29607275 0.34982184 -1.1026102 0.5958487 -0.28598312 0.17209314
-0.39707083 0.26880464 -1.0667359 0.4525643 -0.61781174 0.22608219
